# Autoregressive time series forecasting with the Luna Dataset

This example shows how to train an autoregressive time-series model using the Luna dataset. Autoregressive is based on a very simple idea that observation from the current time steps is decided by the observation from previous steps, and is very suitable for time series data that presents a strong seasonal pattern.

In this notebook you will learn:
- How to transform a univariante-forecasting into a regression problem.
- How to run hyper-parameter optimization and search for the best regression model for your data using built-in `AutoMLExperiment` class."

## Install NuGet packages for training ML.NET models and plotting:

In [ ]:
// using nightly-build
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json"
#r "nuget: Plotly.NET.Interactive, 3.0.2"
#r "nuget: Plotly.NET.CSharp, 0.0.1"
#r "nuget: Microsoft.ML.AutoML, 0.20.0-preview.22458.1"
#r "nuget: Microsoft.Data.Analysis, 0.20.0-preview.22458.1"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/MachineLearning/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.20.0-preview.22458.1 Microsoft.ML.AutoML, 0.20.0-preview.22458.1 Plotly.NET.CSharp, 0.0.1 Plotly.NET.Interactive, 3.0.2

Loading extensions from `Microsoft.ML.AutoML.Interactive.dll`

Loading extensions from `Plotly.NET.Interactive.dll`

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

## Import packages

In [ ]:
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using Microsoft.Data.Analysis;
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.AutoML;
using Microsoft.ML.Data;
using Plotly.NET;

### Import Dataset

Luna is a time-series dataset which records the metric of hourly-active usage of an internal-used service on Azure. It has two columns: `DateTime` and `load`, where `load` column records the metric of hourly-active usage. Luna shows a strong weekly seasonal pattern given its nature as an Azure service, and a slow trend (growth with time) as well. Those features allow you to build a forecasting model to predict the next hour load metric to adjust the size of computing power accordingly.

In the code block below, we show how to
- load dataset
- transform a time-series forecasting problem into a regression problem via using the last _N_ hour as feature and the current `load` as label. We do that because there are more regressors available in ML.Net, which leaves room for AutoML to optimize with. However, based on our practice, you can also achieve pretty close result via sweeping over parameters of `SSA`. 

#### Download or Locate Data
The following code tries to locate the data file in a few known locations or it will download it from the known GitHub location.

In [ ]:
using System;
using System.IO;
using System.Net;

string EnsureDataSetDownloaded(string fileName)
{

	// This is the path if the repo has been checked out.
	var filePath = Path.Combine(Directory.GetCurrentDirectory(),"data", fileName);

	if (!File.Exists(filePath))
	{
		// This is the path if the file has already been downloaded.
		filePath = Path.Combine(Directory.GetCurrentDirectory(), fileName);
	}

	if (!File.Exists(filePath))
	{
		using (var client = new WebClient())
		{
			client.DownloadFile($"https://raw.githubusercontent.com/dotnet/csharp-notebooks/main/machine-learning/data/{fileName}", filePath);
		}
		Console.WriteLine($"Downloaded {fileName}  to : {filePath}");
	}
	else
	{
		Console.WriteLine($"{fileName} found here: {filePath}");
	}

	return filePath;
}

In [ ]:
var dataPath =  EnsureDataSetDownloaded(@"Luna.csv");

// use previous 2 weeks data
var df = DataFrame.LoadCsv(dataPath);
var prevWindows = Enumerable.Range(1, 24 * 7 * 2);
var loads = df["load"].Cast<float?>();
foreach(var i in prevWindows)
{
    var columnName = $"prev_{i}h";
    df[columnName] = DataFrameColumn.Create(columnName, Enumerable.Repeat<float?>(null, i).Concat(loads).SkipLast(i));
}

Luna.csv found here: c:\Users\xiaoyuz\source\repos\csharp-notebooks\machine-learning\data\Luna.csv


### Plotting last three weeks of Luna

In [ ]:
var lastThreeWeek = df["load"].Cast<float>().TakeLast(7 * 24 * 3);
var x = Enumerable.Range(0, lastThreeWeek.Count());
var line = Chart2D.Chart.Line<int, float, string>(x, lastThreeWeek, Name: "load");
line.Display();

<!-- Plotly chart will be drawn inside this DIV -->

### A glance at dataset
We use hourly intervals from the previous 2 weeks as features and current value `load` as label to predict. In doing so, it reframes the forecasting problem into a regression problem, which has more trainer options available in ML.NET. You can either call `Tail(N)` to get the last _N_ rows of data, or `Head(N)` to see the first _N_ rows.

In [ ]:
df.Tail(10)

index,load,prev_1h,prev_2h,prev_3h,prev_4h,prev_5h,prev_6h,prev_7h,prev_8h,prev_9h,prev_10h,prev_11h,prev_12h,prev_13h,prev_14h,prev_15h,prev_16h,prev_17h,prev_18h,prev_19h,prev_20h,prev_21h,prev_22h,prev_23h,prev_24h,prev_25h,prev_26h,prev_27h,prev_28h,prev_29h,prev_30h,prev_31h,prev_32h,prev_33h,prev_34h,prev_35h,prev_36h,prev_37h,prev_38h,prev_39h,prev_40h,prev_41h,prev_42h,prev_43h,prev_44h,prev_45h,prev_46h,prev_47h,prev_48h,prev_49h,prev_50h,prev_51h,prev_52h,prev_53h,prev_54h,prev_55h,prev_56h,prev_57h,prev_58h,prev_59h,prev_60h,prev_61h,prev_62h,prev_63h,prev_64h,prev_65h,prev_66h,prev_67h,prev_68h,prev_69h,prev_70h,prev_71h,prev_72h,prev_73h,prev_74h,prev_75h,prev_76h,prev_77h,prev_78h,prev_79h,prev_80h,prev_81h,prev_82h,prev_83h,prev_84h,prev_85h,prev_86h,prev_87h,prev_88h,prev_89h,prev_90h,prev_91h,prev_92h,prev_93h,prev_94h,prev_95h,prev_96h,prev_97h,prev_98h,prev_99h,prev_100h,prev_101h,prev_102h,prev_103h,prev_104h,prev_105h,prev_106h,prev_107h,prev_108h,prev_109h,prev_110h,prev_111h,prev_112h,prev_113h,prev_114h,prev_115h,prev_116h,prev_117h,prev_118h,prev_119h,prev_120h,prev_121h,prev_122h,prev_123h,prev_124h,prev_125h,prev_126h,prev_127h,prev_128h,prev_129h,prev_130h,prev_131h,prev_132h,prev_133h,prev_134h,prev_135h,prev_136h,prev_137h,prev_138h,prev_139h,prev_140h,prev_141h,prev_142h,prev_143h,prev_144h,prev_145h,prev_146h,prev_147h,prev_148h,prev_149h,prev_150h,prev_151h,prev_152h,prev_153h,prev_154h,prev_155h,prev_156h,prev_157h,prev_158h,prev_159h,prev_160h,prev_161h,prev_162h,prev_163h,prev_164h,prev_165h,prev_166h,prev_167h,prev_168h,prev_169h,prev_170h,prev_171h,prev_172h,prev_173h,prev_174h,prev_175h,prev_176h,prev_177h,prev_178h,prev_179h,prev_180h,prev_181h,prev_182h,prev_183h,prev_184h,prev_185h,prev_186h,prev_187h,prev_188h,prev_189h,prev_190h,prev_191h,prev_192h,prev_193h,prev_194h,prev_195h,prev_196h,prev_197h,prev_198h,prev_199h,prev_200h,prev_201h,prev_202h,prev_203h,prev_204h,prev_205h,prev_206h,prev_207h,prev_208h,prev_209h,prev_210h,prev_211h,prev_212h,prev_213h,prev_214h,prev_215h,prev_216h,prev_217h,prev_218h,prev_219h,prev_220h,prev_221h,prev_222h,prev_223h,prev_224h,prev_225h,prev_226h,prev_227h,prev_228h,prev_229h,prev_230h,prev_231h,prev_232h,prev_233h,prev_234h,prev_235h,prev_236h,prev_237h,prev_238h,prev_239h,prev_240h,prev_241h,prev_242h,prev_243h,prev_244h,prev_245h,prev_246h,prev_247h,prev_248h,prev_249h,prev_250h,prev_251h,prev_252h,prev_253h,prev_254h,prev_255h,prev_256h,prev_257h,prev_258h,prev_259h,prev_260h,prev_261h,prev_262h,prev_263h,prev_264h,prev_265h,prev_266h,prev_267h,prev_268h,prev_269h,prev_270h,prev_271h,prev_272h,prev_273h,prev_274h,prev_275h,prev_276h,prev_277h,prev_278h,prev_279h,prev_280h,prev_281h,prev_282h,prev_283h,prev_284h,prev_285h,prev_286h,prev_287h,prev_288h,prev_289h,prev_290h,prev_291h,prev_292h,prev_293h,prev_294h,prev_295h,prev_296h,prev_297h,prev_298h,prev_299h,prev_300h,prev_301h,prev_302h,prev_303h,prev_304h,prev_305h,prev_306h,prev_307h,prev_308h,prev_309h,prev_310h,prev_311h,prev_312h,prev_313h,prev_314h,prev_315h,prev_316h,prev_317h,prev_318h,prev_319h,prev_320h,prev_321h,prev_322h,prev_323h,prev_324h,prev_325h,prev_326h,prev_327h,prev_328h,prev_329h,prev_330h,prev_331h,prev_332h,prev_333h,prev_334h,prev_335h,prev_336h
0,3388005,3220888,2527278,1856648,1369574,1138552,1092601,1035122,888329,838962,836865,893128,1018963,1246200,1401150,1468952,1552230,2544989,3095399,3427783,3560500,3668727,4004459,4297613,4287130,3879214,2815706,1954505,1413617,1212441,1186555,1110451,1017008,852082,859797,899598,891704,1049837,1207824,1415209,1507603,2411975,2918741,3147765,3290135,3325876,3478702,3786953,3795470,3543817,2559427,1740110,1272833,1082314,1053025,994684,866713,776213,794088,879557,1017037,1152746,1240912,1431362,1536951,2427113,2904028,3133834,3371077,3419092,3578963,3866752,3797630,3460453,2648495,1755543,1248893,1085038,989776,955412,826600,720232,647768,641627,716777,754041,700733,654610,602570,675098,700541,693815,727569,720734,789116,78701

### Construct training pipeline
The following code shows how to construct an ML.NET training pipeline which concatenates all the feature columns with the `prev` prefix into a single feature vector and provides it as input to the regression trainer

Notice that the pipeline calls `mlContext.Auto().Regression` in the end, which returns all available regressors with pre-defined search spaces. Then we can pass this pipeline into `AutoMLExperiment` for hyperparameter optimization.

In [ ]:
var rowCount = df.Rows.Count();
var evaluateCount = 24*7;
var trainDf = df.Head(rowCount -evaluateCount);
var evaluateDf = df.Tail(evaluateCount);
var mlContext = new MLContext();

var featureColumns = df.Columns.Select(c => c.Name)
                        .Where(c => c!="load");
// Append the trainer to the data processing pipeline
var pipeline = mlContext.Transforms.Concatenate(@"Features", featureColumns.ToArray())
                    .Append(mlContext.Auto().Regression(labelColumnName: "load", useLbfgs: false, useSdca: false, useFastForest: false, useFastTree: false, useLgbm: true));

### Run Hyper-parameter optimization using AutoMLExperiment

In [ ]:
// Configure AutoML
var trainTestSplit = mlContext.Data.TrainTestSplit(trainDf, 0.1);

// NotebookMonitor plots trials and show best run nicely in notebook output cell.
var monitor = new NotebookMonitor(pipeline);

var experiment = mlContext.Auto().CreateExperiment()
                    .SetPipeline(pipeline)
                    .SetTrainingTimeInSeconds(60)
                    .SetDataset(trainTestSplit.TrainSet, trainTestSplit.TestSet)
                    .SetRegressionMetric(RegressionMetric.RootMeanSquaredError, "load", "Score")
					.SetMonitor(monitor);

// Configure Visualizer			
monitor.SetUpdate(monitor.Display());

// Start Experiment
var res = await experiment.RunAsync();

index,Trial,Metric,Trainer,Parameters
0,0,285956.6,LightGbmRegression,"{""_pipeline_"":{""_SCHEMA_"":""e0 * e1"",""e0"":{},""e1"":{""NumberOfLeaves"":4,""MinimumExampleCountPerLeaf"":20,""LearningRate"":1,""NumberOfTrees"":4,""SubsampleFraction"":1,""MaximumBinCountPerFeature"":254,""FeatureFraction"":1,""L1Regularization"":2E-10,""L2Regularization"":1,""LabelColumnName"":""load"",""FeatureColumnName"":""Features""}}}"
1,1,170234.92,LightGbmRegression,"{""_pipeline_"":{""_SCHEMA_"":""e0 * e1"",""e0"":{},""e1"":{""NumberOfLeaves"":4,""MinimumExampleCountPerLeaf"":20,""LearningRate"":0.9999997766729865,""NumberOfTrees"":641,""SubsampleFraction"":0.46873628525675043,""MaximumBinCountPerFeature"":310,""FeatureFraction"":0.99999999,""L1Regularization"":7.082058270068566E-10,""L2Regularization"":0.9999997766729865,""LabelColumnName"":""load"",""FeatureColumnName"":""Features""}}}"


## Evaluate model using test dataset
The following code shows how to use the best model produced by hyper-parameter optimization from `AutoMLExperiment` to predict the Luna `load` for next two weeks, then compare it with the actual `load` and calculate regression metric.

In [ ]:
var model = res.Model;
var eval = model.Transform(evaluateDf); // we should use an unseen dataset though.
var metric = mlContext.Regression.Evaluate(eval, "load");
evaluateDf["predicted"] = DataFrameColumn.Create("predicted", eval.GetColumn<float>("Score"));

// print metric
metric

MeanAbsoluteError,MeanSquaredError,RootMeanSquaredError,LossFunction,RSquared
153566.89262462797,38769258423.02824,196899.10721744839,38769258436.76302,0.9690737438487391


## Plot both predicted and truth sonar in test dataset

In [ ]:
var predicted = evaluateDf["predicted"].Cast<float>();
var truth = evaluateDf["load"].Cast<float>();
var X = Enumerable.Range(0, truth.Count());
var predictedChart = Chart2D.Chart.Line<int, float, string>(X, predicted, Name: "predicted");
var truthChart = Chart2D.Chart.Line<int, float, string>(X, truth, Name: "truth");
var combineChart = Chart.Combine(new[]{ predictedChart, truthChart});
combineChart.Display()

<!-- Plotly chart will be drawn inside this DIV -->